In [1]:
%cd D:/uitds

D:\uitds


In [2]:
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [3]:
import json

# read json file
name_not = "v1_epoch2"
name_best1 = "v3_epoch3"
name_best2 = "v2_epoch3"
name_best3 = "v1_1_epoch4"
# vilt_file = "train_vilt_pooler_weighted_CE_9.json"
yes_no_file = f"result_ova/Vintern_test_yes_no_ova_{name_not}.json"
image_ova_file = "result_ova/Vintern_test_image_yes_no_ova_v1_epoch4.json"
text_ova_file = "result_ova/Vintern_test_text_ova_v1_epoch5.json"
multi_file = "result_ova/Vintern_test_multi_yes_no_ova_v1_epoch2.json"

best_result_file = f"result_test_val/test_vintern_{name_best1}.json"
best_result_2_file = f"result_test_val/test_vintern_{name_best2}.json"
best_result_3_file = f"result_test_val/test_vintern_{name_best3}.json"
# image_ova_file = "Vintern_test_image_ova_v1_epoch6_1.json"

with open(image_ova_file) as ova_image,\
        open(text_ova_file) as ova_text,\
        open(best_result_file) as best_result,\
        open(yes_no_file) as yes_no,\
        open(multi_file) as multi_ova,\
        open(best_result_2_file) as best_result_2,\
        open(best_result_3_file) as best_result_3:
    image_ova = json.load(ova_image)
    text_ova = json.load(ova_text)
    yes_no = json.load(yes_no)
    multi_ova = json.load(multi_ova)
    best_result = json.load(best_result)
    best_result_2 = json.load(best_result_2)
    best_result_3 = json.load(best_result_3)

In [4]:
# # read txt file: data\train_val_group_split.txt
# TXT_PATH = "data/train_val_group_split.txt"

# def read_train_test_file(name_file):
    
#     with open(name_file, 'r') as f:
#         lines = f.readlines()
#         num_train, num_val = int(lines[0].split()[0]), int(lines[0].split()[1])
#         train_idx = list(map(int, lines[1].split()))
#         val_idx = list(map(int, lines[2].split()))
#     return num_train, num_val, train_idx, val_idx

# num_train, num_val, train_idx, val_idx = read_train_test_file(TXT_PATH)

# import json
# JSON_PATH = "data/json/vimmsd-train.json"

# json_file = open(JSON_PATH, 'r')
# json_data = json.load(json_file)
# labels = [json_data[str(i)]["label"] for i in val_idx]


In [5]:
import pandas as pd

df = pd.DataFrame({'image_ova': image_ova["results"].values(), 'text_ova': text_ova["results"].values(),'yes_no':yes_no["results"].values(), 'multi_ova': multi_ova["results"].values(),'best_result': best_result["results"].values(),'best_result_2': best_result_2["results"].values(), 'best_result_3': best_result_3["results"].values()})

In [6]:
df.head()

,image_ova,text_ova,yes_no,multi_ova,best_result,best_result_2,best_result_3
0,No,text-sarcasm,Yes,Yes,multi-sarcasm,multi-sarcasm,multi-sarcasm
1,No,text-sarcasm,Yes,Yes,multi-sarcasm,multi-sarcasm,multi-sarcasm
2,No,not-text-sarcasm,No,No,not-sarcasm,not-sarcasm,not-sarcasm
3,No,not-text-sarcasm,Yes,No,multi-sarcasm,multi-sarcasm,not-sarcasm
4,No,not-text-sarcasm,No,No,not-sarcasm,not-sarcasm,multi-sarcasm


In [7]:
def pair_count(*input_dicts):
    """
    Count pairs/groups from multiple input dictionaries
    Args:
        *input_dicts: Variable number of dictionaries, each with a 'results' key
    Returns:
        Dictionary with combined labels as keys and lists of indices as values
    """
    pair_best = {}
    
    # Get the number of results from the first dictionary
    n_results = len(input_dicts[0])
    
    for i in range(n_results):
        # Collect labels from all input dictionaries
        labels = []
        for input_dict in input_dicts:
            labels.append(input_dict[i])
            
        # Create key by joining all labels
        pair_key = " /".join(labels)
        
        # Add to pair_best dictionary
        if pair_key in pair_best:
            pair_best[pair_key].append(i)
        else:
            pair_best[pair_key] = [i]
            
    return pair_best

In [8]:
[(i,len(v)) for (i,v) in pair_count(df["best_result"],df["best_result_2"],df["yes_no"]).items()]

[('multi-sarcasm /multi-sarcasm /Yes', 835),
 ('not-sarcasm /not-sarcasm /No', 338),
 ('not-sarcasm /multi-sarcasm /No', 19),
 ('multi-sarcasm /not-sarcasm /Yes', 77),
 ('not-sarcasm /multi-sarcasm /Yes', 26),
 ('multi-sarcasm /not-sarcasm /No', 28),
 ('not-sarcasm /not-sarcasm /Yes', 59),
 ('multi-sarcasm /image-sarcasm /Yes', 5),
 ('multi-sarcasm /multi-sarcasm /No', 20),
 ('image-sarcasm /multi-sarcasm /Yes', 2),
 ('multi-sarcasm /image-sarcasm /No', 1),
 ('not-sarcasm /image-sarcasm /Yes', 3)]

In [9]:
def old_transform_submit(best_result, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        if best_result[i] == "multi-sarcasm" and yes_no[i] == "No":
            transformed_labels[i] = "not-sarcasm"
        elif best_result[i] == "not-sarcasm" and yes_no[i] == "Yes":
            transformed_labels[i] = "multi-sarcasm"
    return transformed_labels

In [10]:
def transform_submit(best_result, best_result_2,best_result_3, yes_no):
    transformed_labels = []
    for i in range(len(best_result)):
        transformed_labels.append(best_result[i])
        # image-sarcasm
        #     not-sarcasm not-sarcasm image-sarcasm Yes: 2
        #     not-sarcasm multi-sarcasm image-sarcasm Yes: 1
        if(best_result[i] == "not-sarcasm" and best_result_2[i] == "not-sarcasm" and best_result_3[i] == "image-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "image-sarcasm" 
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "multi-sarcasm" and best_result_3[i] == "image-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "image-sarcasm"
        # not-sarcasm
        #     multi-sarcasm not-sarcasm multi-sarcasm No: 13
        #     multi-sarcasm not-sarcasm not-sarcasm No: 29
        #     multi-sarcasm multi-sarcasm multi-sarcasm No: 13
        #     image-sarcasm not-sarcasm image-sarcasm No: 2
        #     multi-sarcasm multi-sarcasm not-sarcasm No: 9 
        elif(best_result[i] == "multi-sarcasm" and best_result_2[i] == "not-sarcasm" and best_result_3[i] == "multi-sarcasm" and yes_no[i] == "No"):
            transformed_labels[i] = "not-sarcasm"
        elif(best_result[i] == "multi-sarcasm" and best_result_2[i] == "not-sarcasm" and best_result_3[i] == "not-sarcasm" and yes_no[i] == "No"):
            transformed_labels[i] = "not-sarcasm"
        elif(best_result[i] == "multi-sarcasm" and best_result_2[i] == "multi-sarcasm" and best_result_3[i] == "multi-sarcasm" and yes_no[i] == "No"):
            transformed_labels[i] = "not-sarcasm"
        elif(best_result[i] == "image-sarcasm" and best_result_2[i] == "not-sarcasm" and best_result_3[i] == "image-sarcasm" and yes_no[i] == "No"):
            transformed_labels[i] = "not-sarcasm"
        elif(best_result[i] == "multi-sarcasm" and best_result_2[i] == "multi-sarcasm" and best_result_3[i] == "not-sarcasm" and yes_no[i] == "No"):
            transformed_labels[i] = "not-sarcasm"
        # multi-sarcasm
        #     not-sarcasm not-sarcasm multi-sarcasm Yes: 14
        #     not-sarcasm multi-sarcasm multi-sarcasm Yes: 29
        #     not-sarcasm not-sarcasm not-sarcasm Yes: 31
        #     not-sarcasm multi-sarcasm not-sarcasm Yes: 18
        #     not-sarcasm image-sarcasm multi-sarcasm Yes: 1
        #     not-sarcasm image-sarcasm image-sarcasm Yes: 1
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "not-sarcasm" and best_result_3[i] == "multi-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "multi-sarcasm"
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "multi-sarcasm" and best_result_3[i] == "multi-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "multi-sarcasm"
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "not-sarcasm" and best_result_3[i] == "not-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "multi-sarcasm"
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "multi-sarcasm" and best_result_3[i] == "not-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "multi-sarcasm"
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "image-sarcasm" and best_result_3[i] == "multi-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "multi-sarcasm"
        elif(best_result[i] == "not-sarcasm" and best_result_2[i] == "image-sarcasm" and best_result_3[i] == "image-sarcasm" and yes_no[i] == "Yes"):
            transformed_labels[i] = "multi-sarcasm"
    return transformed_labels

In [11]:
transformed_labels = transform_submit(df["best_result"], df["best_result_2"],df["best_result_3"],df["yes_no"])

In [12]:
old_transformed_labels = old_transform_submit(df["best_result"], df["yes_no"])

In [13]:
from collections import Counter

In [14]:
Counter(df["best_result"])

Counter({'multi-sarcasm': 966, 'not-sarcasm': 445, 'image-sarcasm': 2})

In [15]:
Counter(old_transformed_labels)

Counter({'multi-sarcasm': 1005, 'not-sarcasm': 406, 'image-sarcasm': 2})

In [16]:
yes_no_epoch2_results = {}
yes_no_epoch2_results["results"] = {}
yes_no_epoch2_results["phase"] = "dev"
len(best_result["results"]), len(yes_no_epoch2_results["results"])
for i in range(len(transformed_labels)):
    yes_no_epoch2_results["results"][str(i)] = transformed_labels[i]
Counter(yes_no_epoch2_results["results"].values())

Counter({'multi-sarcasm': 1002, 'not-sarcasm': 405, 'image-sarcasm': 6})

In [17]:
a = b 

NameError: name 'b' is not defined

In [ ]:
# save the results
with open(f"merge_val/best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}_results.json", "w") as out:
    json.dump(yes_no_epoch2_results, out)
# save the results
# with open(f"merge_val/best_{name_best1}_not_{name_2}_results.json", "w") as out:
#     json.dump(yes_no_epoch2_results, out)

In [ ]:
f"best_{name_best1}_best2_{name_best2}_best3_{name_best3}_not_{name_not}"